# Fase de Análisis Inicial y Unión de Datos

En esta fase el objetivo será analizar todos los conjuntos de datos que tenemos, identificar sus problemas y desarrollar una serie de funciones para solucionar problemas que no nos permiten unir los datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
sys.path.append("../") #Para poder importar
from src.SupportExploracion import traducir, fill_year, year_to_int, date_to_datetime, numeric_conversion,fill_organo_superior
warnings.filterwarnings('ignore')
# Carga el dataset de trabajo
pd.set_option("display.max_columns", None) #Ver todo
df_2013 = pd.read_csv("../datos/raw/datos-2013.csv",delimiter=";")
df_2014 = pd.read_csv("../datos/raw/datos-2014.csv",delimiter=";")   
df_2015 = pd.read_csv("../datos/raw/datos-2015.csv",delimiter=";")
df_2016 = pd.read_csv("../datos/raw/datos-2016.csv",delimiter=";")
df_2017 = pd.read_csv("../datos/raw/datos-2017.csv",delimiter=";")
df_2018 = pd.read_csv("../datos/raw/datos-2018.csv",delimiter=";")
df_2019 = pd.read_csv("../datos/raw/datos-2019.csv",delimiter=";")
df_2020 = pd.read_csv("../datos/raw/datos-2020.csv",delimiter=";")
df_2021 = pd.read_csv("../datos/raw/datos-2021.csv",delimiter=";")

En la carga de datos observamos que el delimitador usado es el ; en vez de la ,

In [2]:
columnas = list(df_2013.columns)
for columna in columnas:
    newname = traducir(columna) #Devuelve un diccionario
    df_2013.rename(columns=newname,inplace=True)
    df_2014.rename(columns=newname,inplace=True)
    df_2015.rename(columns=newname,inplace=True)
    df_2016.rename(columns=newname,inplace=True)
    df_2017.rename(columns=newname,inplace=True)
    df_2018.rename(columns=newname,inplace=True)
    df_2019.rename(columns=newname,inplace=True)
    df_2020.rename(columns=newname,inplace=True)
    df_2021.rename(columns=newname,inplace=True)
    

Realizamos la conversion de todas las columnas de todas las tablas al español y minúscula

# Fix año Fiscal

Rellenamos en cada Dataframe la columna año fiscal, para dejar la columna con el menor número de nulos posibles y la volvemos tipo int

In [3]:
lista_df = [df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021]
years = [2013,2014,2015,2016,2017,2018,2019,2020,2021]
rellenaryear = list(map(fill_year,lista_df,years))
pasarint = list(map(year_to_int,lista_df))


# Pasar fecha de registro a formato datetime

Para tener todos los datos en el mismo formato vamos a pasar todas las columnas "fecha_registro" a formato datetime ignorando los valores nulos

In [4]:
lista_df = [df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021]
todatetime = list(map(date_to_datetime,lista_df))
df_2017.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190479 entries, 0 to 190478
Data columns (total 16 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   código del órgano superior   185999 non-null  float64       
 1   nombre del órgano superior   123811 non-null  object        
 2   código del órgano            188418 non-null  float64       
 3   nombre del órgano            185457 non-null  object        
 4   código de la unidad gestora  186949 non-null  float64       
 5   nombre de la unidad gestora  188076 non-null  object        
 6   categoría económica          187950 non-null  object        
 7   origen de los ingresos       181808 non-null  object        
 8   tipo de ingresos             181798 non-null  object        
 9   detalle                      186726 non-null  object        
 10  valor previsto actualizado   180955 non-null  object        
 11  valor registrado          

In [5]:
datos = df_2013.groupby("código del órgano superior")["nombre del órgano superior"].first()
lista_codigos = list(datos.index)
lista_names = list(datos.values)
diccionario = {}
for i in range(0, len(lista_codigos)):
    diccionario[lista_codigos[i]] = lista_names[i]    
    

In [6]:
df_2013.head(1)

,código del órgano superior,nombre del órgano superior,código del órgano,nombre del órgano,código de la unidad gestora,nombre de la unidad gestora,categoría económica,origen de los ingresos,tipo de ingresos,detalle,valor previsto actualizado,valor registrado,valor ejecutado,porcentaje ejecutado,fecha de registro,año fiscal
0,63000.0,NaN,63000.0,Advocacia-Geral da União - Unidades com víncul...,110060.0,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,"0,00","0,00","1297,13","0,00",2013-12-31,2013


# Convertir valores a numéricos
Observamos que las columnas:
- valor previsto actualizado
- valor registrado
- valor ejecutado
- porcentaje ejecutado

son de tipo object, vamos a convertirlas en numéricas en todos los campos.

In [7]:
pd.set_option('display.float_format', '{:.2f}'.format) #Usamos esto porque se nos mostraba en notación científica el valor ejecutado
lista_df = [df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021]
tonumeric = list(map(numeric_conversion,lista_df))



# Rellenar datos faltantes en nombre del órgano superior

Otra observación realizada fue que el código del órgano superior y que del nombre el órgano superior, tenían muchos valores nulos, pero ví que el código era identificativo del nombre, por lo que he desarrollado un diccionario de valores únicos, lo que me ha permitido rellenar nombre del órgano superior por código único, reduciendo significativamente los valores nulos del mismo.

La razón de hacerlo en esta fase consiste en la facilidad de trabajar con los datos separados, aunque requiere realizar las operaciones una por una con el map, nos va a permitir tener una tabla total mucho más completa en datos, para luego poder limpiarla y analizar de forma mucho más sencilla


In [8]:
lista_df = [df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019,df_2020,df_2021]
fillOrgGobierno = list(map(fill_organo_superior,lista_df))

In [9]:
condicion = df_2013["nombre del órgano superior"].isnull()
df_2013[condicion]

,código del órgano superior,nombre del órgano superior,código del órgano,nombre del órgano,código de la unidad gestora,nombre de la unidad gestora,categoría económica,origen de los ingresos,tipo de ingresos,detalle,valor previsto actualizado,valor registrado,valor ejecutado,porcentaje ejecutado,fecha de registro,año fiscal
31,NaN,NaN,22905.00,NaN,130137.00,FUNDO DE DEFESA DA ECON.CAFEEIRA/FUNCAFE/SPA,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,61810.00,0.00,0.00,0.00,2013-12-31,2013
75,NaN,NaN,44208.00,Serviço Florestal Brasileiro,440088.00,SETORIAL PROG.ORCAMENTARIA E FINANCEIRA-SFB,Receitas Correntes,Outras Receitas Correntes,NaN,OUTRAS RECEITAS,NaN,0.00,2143.42,0.00,2013-12-31,2013
243,NaN,NaN,24209.00,Centro Nacional de Tecnologia Eletrônica Avanç...,245209.00,CENTRO NACIONAL DE TECNOLOGIA ELETRONICA AVAN,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.00,0.00,140492.88,0.00,2013-12-31,2013
282,NaN,NaN,24000.00,"Ministério da Ciência, Tecnologia, Inovações e...",240102.00,COORDENACAO-GERAL DE ORCAMENTO E FINANCAS,Receitas Correntes,Receita de Serviços,Receita de Serviços,NaN,38107.00,0.00,0.00,0.00,2013-12-31,2013
301,NaN,NaN,24000.00,"Ministério da Ciência, Tecnologia, Inovações e...",240102.00,COORDENACAO-GERAL DE ORCAMENTO E FINANCAS,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MULTAS E JUROS PREVISTOS EM CONTRATOS,60695.00,0.00,495673.26,81700.00,2013-12-31,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4302,NaN,NaN,44000.00,Ministério do Meio Ambiente - Unidades com vín...,440002.00,SUBSECRET.PLANEJAM.ORCAM.E ADMINIST±SPOA(MMA),Receitas Correntes,Transferências Correntes,Transferências de Instituições Privadas,Transferências de Instituições Privadas,4999776.00,0.00,1169490.94,2300.00,2013-12-31,2013
4406,NaN,NaN,20408.00,Fundação Cultural Palmares,344041.00,FUNDACAO CULTURAL PALMARES,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RESTITUICOES DE CONVENIOS,0.00,0.00,61893.43,0.00,2013-12-31,2013
4426,NaN,NaN,NaN,Agência Nacional do Cinema,203003.00,AGENCIA NACIONAL DO CINEMA,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS,0.00,0.00,233.03,0.00,2013-12-31,2013
4462,NaN,NaN,20101.00,Presidência da República,110005.00,SECRETARIA DE ADMINISTRACAO-SET.ORC.E FIN./PR,Receitas Correntes,Outras Receitas Correntes,Demais receitas correntes,VARIACAO CAMBIAL,0.00,0.00,292.34,0.00,2013-12-31,2013


# Terminar de darle valor a las columnas
Además del análisis previo esto me revela que los valores únicos restantes son aquellos que tienen nulos en el código y nombre del órgano superior, podemos encontrar dichos valores gracias a código del órgano ya que si nos fijamos es un subcódigo, es decir 63

In [10]:
df_2013

,código del órgano superior,nombre del órgano superior,código del órgano,nombre del órgano,código de la unidad gestora,nombre de la unidad gestora,categoría económica,origen de los ingresos,tipo de ingresos,detalle,valor previsto actualizado,valor registrado,valor ejecutado,porcentaje ejecutado,fecha de registro,año fiscal
0,63000.00,Advocacia-Geral da União,63000.00,Advocacia-Geral da União - Unidades com víncul...,110060.00,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIVIDA ATIVA NAO TRIBUTARIA DE OUTRAS REC,0.00,0.00,1297.13,0.00,2013-12-31,2013
1,63000.00,Advocacia-Geral da União,63000.00,Advocacia-Geral da União - Unidades com víncul...,110060.00,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",RECUPERACAO DE DESPESAS DE EXERC. ANTERIORES,0.00,0.00,26666621.42,0.00,2013-12-31,2013
2,63000.00,Advocacia-Geral da União,63000.00,Advocacia-Geral da União - Unidades com víncul...,110060.00,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,0.00,0.00,301251.13,0.00,2013-12-31,2013
3,63000.00,Advocacia-Geral da União,63000.00,Advocacia-Geral da União - Unidades com víncul...,110060.00,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Bens, Direitos e Valores Incorporados ao Patr",REC.DIV.ATIVA POR INFRAÇÃO ADMINISTRATIVA,0.00,0.00,1855.58,0.00,2013-12-31,2013
4,63000.00,Advocacia-Geral da União,63000.00,Advocacia-Geral da União - Unidades com víncul...,110060.00,COORD. GERAL DE ORC. FIN. E ANAL. CONT. - AGU,Receitas Correntes,Outras Receitas Correntes,"Indenizações, restituições e ressarcimentos",OUTRAS RESTITUICOES,0.00,0.00,52140.68,0.00,2013-12-31,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4493,20000.00,Presidência da República,20101.00,Presidência da República,110005.00,SECRETARIA DE ADMINISTRACAO-SET.ORC.E FIN./PR,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS,0.00,0.00,8.34,0.00,2013-12-31,2013
4494,20000.00,Presidência da República,64000.00,NaN,200016.00,SECRETARIA DE DIREITOS HUMANOS/PR,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS,0.00,0.00,0.28,0.00,2013-12-31,2013
4495,20000.00,Presidência da República,20101.00,Presidência da República,110005.00,SECRETARIA DE ADMINISTRACAO-SET.ORC.E FIN./PR,Receitas Correntes,Receita Patrimonial,Exploração do patrimônio imobiliário do Estad,ALUGUEIS,0.00,0.00,61593.03,0.00,2013-12-31,2013
4496,20000.00,Presidência da República,20116.00,Fundo de Imprensa Nacional,110245.00,FUNDO DE IMPRENSA NACIONAL/EXEC.ORC.FINANC.,Receitas Correntes - intra-orçamentárias,Receita Industrial,Receitas da indústria de transformação,RECEITA DA INDUSTRIA EDITORIAL E GRAFICA,120839273.00,0.00,161286016.04,13300.00,2013-12-31,2013


In [12]:
condicion = df_2013["código del órgano"].isnull()
df_2013[condicion]

,código del órgano superior,nombre del órgano superior,código del órgano,nombre del órgano,código de la unidad gestora,nombre de la unidad gestora,categoría económica,origen de los ingresos,tipo de ingresos,detalle,valor previsto actualizado,valor registrado,valor ejecutado,porcentaje ejecutado,fecha de registro,año fiscal
23,22000.00,"Ministério da Agricultura, Pecuária e Abastec",NaN,"Ministério da Agricultura, Pecuária e Abasteci...",130101.00,COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA,Receitas Correntes,Receita Agropecuária,Receita da produção animal e derivados,Receita da produção animal e derivados,350042.00,0.00,107548.62,3100.00,2013-12-31,2013
37,22000.00,"Ministério da Agricultura, Pecuária e Abastec",NaN,Fundo de Defesa da Economia Cafeeira,130137.00,FUNDO DE DEFESA DA ECON.CAFEEIRA/FUNCAFE/SPA,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS E JUROS DE MORA,20149.00,0.00,548.49,300.00,2013-12-31,2013
87,22000.00,"Ministério da Agricultura, Pecuária e Abastec",NaN,"Ministério da Agricultura, Pecuária e Abasteci...",130101.00,COORD.-GERAL DE ORCAMENTO E FINANCAS-MAPA,Receitas Correntes,Receita Patrimonial,Exploração do patrimônio imobiliário do Estad,TAXA DE OCUPACAO DE TERRENOS DA UNIAO,22604.00,0.00,20063.35,8900.00,2013-12-31,2013
92,22000.00,"Ministério da Agricultura, Pecuária e Abastec",NaN,Empresa Brasileira de Pesquisa Agropecuária,135037.00,NaN,Receitas Correntes,Receita Agropecuária,Outras receitas agropecuárias,Outras receitas agropecuárias,0.00,0.00,375.00,0.00,2013-12-31,2013
118,22000.00,"Ministério da Agricultura, Pecuária e Abastec",NaN,Companhia Nacional de Abastecimento,NaN,COMPANHIA NACIONAL DE ABASTECIMENTO,Receitas de Capital,Alienação de Bens,Alienação de bens móveis,ALIENACAO DE EQUIPAMENTOS,NaN,0.00,NaN,0.00,2013-12-31,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4413,54000.00,Ministério do Turismo,NaN,Fundação Biblioteca Nacional,344042.00,FUNDACAO BIBLIOTECA NACIONAL,Receitas Correntes,Receita de Serviços,Receita de Serviços,OUTROS RECEITAS DE SERVICOS,10704.00,0.00,0.00,0.00,2013-12-31,2013
4418,54000.00,Ministério do Turismo,NaN,Fundo Nacional de Cultura,340002.00,COORD.GERAL DE PLAN. ORC. FIN. E CONTAB/FNC,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",MUL.JUR.MORA CONTR.S/REC.SORTEIOS ENT.FILANTR,9814.00,NaN,0.00,0.00,2013-12-31,2013
4426,NaN,NaN,NaN,Agência Nacional do Cinema,203003.00,AGENCIA NACIONAL DO CINEMA,Receitas Correntes,Outras Receitas Correntes,"Multas administrativas, contratuais e judicia",OUTRAS MULTAS,0.00,0.00,233.03,0.00,2013-12-31,2013
4448,54000.00,Ministério do Turismo,NaN,Fundo Nacional de Cultura,340002.00,NaN,Receitas Correntes,Contribuições,Contribuições sociais,CONTRIBUICAO S/ARREC.FUNDOS INVEST.REGIONAIS,76401.00,0.00,NaN,0.00,2013-12-31,2013
